In [6]:
import snowflake.connector


In [31]:
table = 'EXPENDITURE_V2'
con = snowflake.connector.connect(
    user="NAVKARANRANDHAWA",  # given on your instruction email
    password="Adviseinc1!",  # set by you after you log in to Snowflake for the first time
    account="LS54927.eu-west-2.aws",
    role="navkaranrandhawa",  # this would've been created by us, it will be the same as your user but all lower case
    database="NAVKARANRANDHAWA",  # you will see a database with your name when you log in
    warehouse="compute_wh",
    schema = "test schema",
    table = table,
    session_parameters={
        "QUERY_TAG": "data-recruitment",
    },
)

['Department Family', 'Entity', 'Date', 'Expense Type', 'Expense Area', 'Supplier', 'Transaction Number', 'Expenditure']


/var/folders/mk/fsds1mrn35jdn5l23w6gvph00000gp/T/ipykernel_81335/3541449761.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [56]:
schema = "EXPENDITURE_V2_SCHEMA"

con = snowflake.connector.connect(
    user="NAVKARANRANDHAWA",  # given on your instruction email
    password="Adviseinc1!",  # set by you after you log in to Snowflake for the first time
    account="LS54927.eu-west-2.aws",
    role="navkaranrandhawa",  # this would've been created by us, it will be the same as your user but all lower case
    database="NAVKARANRANDHAWA",  # you will see a database with your name when you log in
    warehouse="compute_wh",
    schema = schema,
    session_parameters={
        "QUERY_TAG": "data-recruitment",
    },)
    

In [66]:
cursor = con.cursor()
create_schema_query = f"CREATE SCHEMA IF NOT EXISTS {schema}"
cursor.execute(create_schema_query)

create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {schema}.{table} (
        "Department Family" STRING,
        "Entity" STRING,
        "Date" DATE,
        "Expense Type" STRING,
        "Expense Area" STRING,
        "Supplier" STRING,
        "Transaction Number" STRING,
        "Expenditure" FLOAT
    )
"""
cursor.execute(create_table_query)
con.commit()

In [82]:
csv_file_path = '/Users/navkaranrandhawa/Documents/Personal/adviseinc/data-recruitment/expenditure_v2.csv'
stage_name = f"{schema}_STAGE"
stage_csv_query = f"""
    CREATE STAGE IF NOT EXISTS {stage_name}
    FILE_FORMAT = (
        TYPE = 'CSV'
        FIELD_DELIMITER = ','
        SKIP_HEADER = 1
    )
"""
cursor.execute(stage_csv_query)

In [83]:
copy_into_query = f"""
    COPY INTO {table}
    FROM @{stage_name}
    FILE_FORMAT = (
        TYPE = 'CSV'
        FIELD_DELIMITER = ','
        SKIP_HEADER = 1
    )
"""
cursor.execute(copy_into_query)
con.commit()

In [86]:
put_csv_query = f"""
    PUT file://{csv_file_path} @{stage_name}
"""
cursor.execute(put_csv_query)
con.commit()

In [ ]:
SQL Queries
Distinct 
SELECT COUNT(DISTINCT "Transaction Number") AS transaction_count
FROM EXPENDITURE_V2_SCHEMA.EXPENDITURE_V2 ;

In [ ]:
SELECT EXTRACT(YEAR FROM "Date") AS Year,
       AVG("Expenditure") AS AverageYearlySpend
FROM EXPENDITURE_V2_SCHEMA.EXPENDITURE_V2
WHERE "Entity" = 'Salisbury NHS Foundation Trust'
GROUP BY Year;